In [69]:
from utils import get_mongo_client, test_mongo_client
from bson.timestamp import Timestamp
from datetime import datetime
TARGET_ENV = "production" #"backup"

client_prod = get_mongo_client("production")
test_mongo_client(client_prod)
client_backup = get_mongo_client("backup")
test_mongo_client(client_backup)
client_local = get_mongo_client("local")
test_mongo_client(client_local)

Pinged your deployment. You successfully connected to MongoDB!
Pinged your deployment. You successfully connected to MongoDB!
Pinged your deployment. You successfully connected to MongoDB!


In [72]:
oplog = client_backup["local"]['oplog.rs']

# Specify your cutoff timestamp (year, month, day, hour, minute, second)
cutoff_timestamp = datetime(2023, 1, 12, 15, 0, 0)  # Example: January 1, 2023, at 00:00:00
cutoff_bson_timestamp = Timestamp(int(cutoff_timestamp.timestamp()), 1)

#last_write_op = oplog.find({'op': {'$in': ['i', 'u', 'd']}, 'ns': {'$not': {'$regex': '^config\\.'}}}).sort('$natural', -1).limit(1)
# Find the last write operation before the cutoff timestamp
last_write_op = oplog.find({'op': {'$in': ['i', 'u', 'd']},
                            'ts': {'$lt': cutoff_bson_timestamp},
                            'ns': {'$not': {'$regex': '^config\\.'}}}).sort('$natural', -1).limit(1)

for op in last_write_op:
        timestamp = op['ts'].as_datetime()
        operation_type = op['op']
        namespace = op['ns']
        print("Timestamp of the last write operation:", timestamp)
        print("Operation type:", operation_type)
        print("Namespace (database.collection):", namespace)